***GENERATED CODE FOR customerchurnfe PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
from hdfs3 import HDFileSystem
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs = HDFileSystem(
            host=eval(config)['host'], port=eval(config)['port'])
        with hdfs.open(eval(config)['url']) as f:
            df = pd.read_csv(f, error_bad_lines=False)
        df = spark.createDataFrame(dfPd)
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.ml.feature import Binarizer
from pyspark.sql.functions import round
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


def BinarizerTransform(df, params, transformationData={}):
    dfReturn = df
    transform_params = params
    feature = transform_params['feature']
    outcol = feature + "_binarizer"
    dfReturn = dfReturn.withColumn("feature_cast", dfReturn[feature].cast("double")).drop(feature)\
        .withColumnRenamed("feature_cast", feature)

    dfReturn = dfReturn.fillna({feature: 0.0})
    binarizer = Binarizer(threshold=float(
        transformationData['threshold']), inputCol=feature, outputCol=outcol)
    binarizedDataFrame = binarizer.transform(dfReturn)

    # binarizedDataFrame=binarizedDataFrame.drop(feature).withColumnRenamed(outcol,feature)

    dfReturn = binarizedDataFrame
    dfReturn = dfReturn.withColumn(feature, round(dfReturn[feature], 2))

    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Surname', 'transformation_label': 'String Indexer'}], 'feature': 'Surname', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': 'Ahmed', 'max': 'Zetticci', 'missing': '0', 'distinct': '418'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Surname'}, {'feature_label': 'Surname', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Surname')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Geography', 'transformation_label': 'String Indexer'}], 'feature': 'Geography', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'France', 'max': 'Spain', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Geography'}, {'feature_label': 'Geography', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Geography')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Gender', 'transformation_label': 'String Indexer'}], 'feature': 'Gender', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Female', 'max': 'Male', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Gender'}, {'feature_label': 'Gender', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Gender')
        transformationDF = BinarizerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Balance', 'threshold': 74362.842, 'transformation_label': 'Binarizer'}], 'feature': 'Balance', 'type': 'real', 'selected': 'True', 'replaceby': 'mean', 'stats': {
            'count': '500', 'mean': '74363.15', 'stddev': '62553.58', 'min': '0.0', 'max': '213146.2', 'missing': '0'}, 'transformation': [{'transformation': 'Binarizer', 'selectedAsDefault': 1}], 'updatedLabel': 'Balance'}, {'feature_label': 'Balance', 'threshold': 74362.842, 'transformation_label': 'Binarizer'})
        transformationDF = transformationDF.drop('Balance')
        transformationDF = BinarizerTransform(transformationDF, {'transformationsData': [{'feature_label': 'EstimatedSalary', 'threshold': 103007.368, 'transformation_label': 'Binarizer'}], 'feature': 'EstimatedSalary', 'type': 'real', 'selected': 'True', 'replaceby': 'mean', 'stats': {
            'count': '500', 'mean': '103007.86', 'stddev': '57581.16', 'min': '371.05', 'max': '199725.39', 'missing': '0'}, 'transformation': [{'transformation': 'Binarizer', 'selectedAsDefault': 1}], 'updatedLabel': 'EstimatedSalary'}, {'feature_label': 'EstimatedSalary', 'threshold': 103007.368, 'transformation_label': 'Binarizer'})
        transformationDF = transformationDF.drop('EstimatedSalary')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Card Type', 'transformation_label': 'String Indexer'}], 'feature': 'Card Type', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'DIAMOND', 'max': 'SILVER', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Card Type'}, {'feature_label': 'Card Type', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Card Type')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***CONNECTOR FUNCTIONS TO WRITE DATA.***

In [ ]:
import datetime
import requests
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class NumtraConnector:

    def put(inStages, inStagesData, stageId, spark, config):
        path = eval(config)['server_url']
        baseType = eval(config)['baseType']
        results_url = eval(config)['results_url']
        server = eval(config)['server']
        originalfile = eval(config)['orignalKey']
        eval(config)['pathOnly']
        filename = eval(config)['filename']
        eval(config)['ser']
        eval(config)['user']
        eval(config)['password']
        eval(config)['authSource']
        eval(config)['user_id']
        eval(config)['parent_id']
        eval(config)['project_id']
        time = str(int(datetime.datetime.now().timestamp()))

        inStagesData[inStages[0]]

        print(path)
        print(baseType)
        print(results_url)
        print(server)
        print(originalfile)
        print(filename)

        args = {
            'url': path,
            'baseType': baseType,
            'originalfile': originalfile,
            'filename': time + filename
        }

        response = requests.post(results_url, args)
        return response


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('spark://backendmaster:7077').getOrCreate()
#%run customerchurnfeHooks.ipynb
try:
	#sourcePreExecutionHook()

	customerchurnrecords = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/Customer-Churn-Records.csv', 'filename': 'Customer-Churn-Records.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'dbfs_token': '', 'dbfs_domain': '', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/', 'results_url': 'http://colaberry.numtra.com:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run customerchurnfeHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(customerchurnrecords,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "RowNumber", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "250.5", "stddev": "144.48", "min": "1", "max": "500", "missing": "0"}, "updatedLabel": "RowNumber"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CustomerId", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "15693572.05", "stddev": "70877.62", "min": "15566111", "max": "15813844", "missing": "0"}, "updatedLabel": "CustomerId"}, {"transformationsData": [{"feature_label": "Surname", "transformation_label": "String Indexer"}], "feature": "Surname", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Ahmed", "max": "Zetticci", "missing": "0", "distinct": "418"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Surname"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CreditScore", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "647.4", "stddev": "101.14", "min": "376", "max": "850", "missing": "0"}, "updatedLabel": "CreditScore"}, {"transformationsData": [{"feature_label": "Geography", "transformation_label": "String Indexer"}], "feature": "Geography", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "France", "max": "Spain", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Geography"}, {"transformationsData": [{"feature_label": "Gender", "transformation_label": "String Indexer"}], "feature": "Gender", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Female", "max": "Male", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Gender"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Age", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "38.11", "stddev": "9.96", "min": "19", "max": "80", "missing": "0"}, "updatedLabel": "Age"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Tenure", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "5.13", "stddev": "2.99", "min": "0", "max": "10", "missing": "0"}, "updatedLabel": "Tenure"}, {"transformationsData": [{"feature_label": "Balance", "threshold": 74362.842, "transformation_label": "Binarizer"}], "feature": "Balance", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "74363.15", "stddev": "62553.58", "min": "0.0", "max": "213146.2", "missing": "0"}, "transformation": [{"transformation": "Binarizer", "selectedAsDefault": 1}], "updatedLabel": "Balance"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "NumOfProducts", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1.53", "stddev": "0.57", "min": "1", "max": "4", "missing": "0"}, "updatedLabel": "NumOfProducts"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "HasCrCard", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.71", "stddev": "0.46", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "HasCrCard"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "IsActiveMember", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.49", "stddev": "0.5", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "IsActiveMember"}, {"transformationsData": [{"feature_label": "EstimatedSalary", "threshold": 103007.368, "transformation_label": "Binarizer"}], "feature": "EstimatedSalary", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "103007.86", "stddev": "57581.16", "min": "371.05", "max": "199725.39", "missing": "0"}, "transformation": [{"transformation": "Binarizer", "selectedAsDefault": 1}], "updatedLabel": "EstimatedSalary"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Exited", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.2", "stddev": "0.4", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Exited"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Complain", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.21", "stddev": "0.41", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Complain"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Satisfaction Score", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "3.03", "stddev": "1.34", "min": "1", "max": "5", "missing": "0"}, "updatedLabel": "Satisfaction Score"}, {"transformationsData": [{"feature_label": "Card Type", "transformation_label": "String Indexer"}], "feature": "Card Type", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "DIAMOND", "max": "SILVER", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Card Type"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Point Earned", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "595.12", "stddev": "227.38", "min": "119", "max": "994", "missing": "0"}, "updatedLabel": "Point Earned"}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***WRITING DATAFRAME***

In [ ]:
#%run customerchurnfeHooks.ipynb
try:
	#sinkPreExecutionHook()

	bankcustomerchurnoutput = NumtraConnector.fetch(spark, "{'samplefile': '/FileStore/platform/sampleData/6563b72a3abc3aea7a00badc/part-00000-7ac7a034-8987-4edb-a148-e2e3d69e38a9-c000.csv', 'samplecount': 977, 'originalcount': 10000, 'orignalKey': '/FileStore/platform/uploadedSourceFiles/Customer-Churn-Records.csv', 'pathOnly': '/Bank Customer Churn', 'project_id': '6563b6be3abc3aea7a00bac5', 'parent_id': '6563b6be3abc3aea7a00bac5', 'original_schema': [{'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6563b8cb3abc3aea7a00bb36', 'field': 'RowNumber', 'alias': 'RowNumber', 'type': 'numeric', 'position': '0'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6563b8cb3abc3aea7a00bb37', 'field': 'CustomerId', 'alias': 'CustomerId', 'type': 'numeric', 'position': '1'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6563b8cb3abc3aea7a00bb38', 'field': 'CreditScore', 'alias': 'CreditScore', 'type': 'numeric', 'position': '2'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6563b8cb3abc3aea7a00bb39', 'field': 'Age', 'alias': 'Age', 'type': 'numeric', 'position': '3'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6563b8cb3abc3aea7a00bb3a', 'field': 'Tenure', 'alias': 'Tenure', 'type': 'numeric', 'position': '4'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6563b8cb3abc3aea7a00bb3b', 'field': 'NumOfProducts', 'alias': 'NumOfProducts', 'type': 'numeric', 'position': '5'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6563b8cb3abc3aea7a00bb3c', 'field': 'HasCrCard', 'alias': 'HasCrCard', 'type': 'numeric', 'position': '6'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6563b8cb3abc3aea7a00bb3d', 'field': 'IsActiveMember', 'alias': 'IsActiveMember', 'type': 'numeric', 'position': '7'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6563b8cb3abc3aea7a00bb3e', 'field': 'Exited', 'alias': 'Exited', 'type': 'numeric', 'position': '8'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6563b8cb3abc3aea7a00bb3f', 'field': 'Complain', 'alias': 'Complain', 'type': 'numeric', 'position': '9'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6563b8cb3abc3aea7a00bb40', 'field': 'Satisfaction Score', 'alias': 'Satisfaction Score', 'type': 'numeric', 'position': '10'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6563b8cb3abc3aea7a00bb41', 'field': 'Point Earned', 'alias': 'Point Earned', 'type': 'numeric', 'position': '11'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6563b8cb3abc3aea7a00bb42', 'field': 'Surname_stringindexer', 'alias': 'Surname_stringindexer', 'type': 'real', 'position': '12'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6563b8cb3abc3aea7a00bb43', 'field': 'Geography_stringindexer', 'alias': 'Geography_stringindexer', 'type': 'numeric', 'position': '13'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6563b8cb3abc3aea7a00bb44', 'field': 'Gender_stringindexer', 'alias': 'Gender_stringindexer', 'type': 'numeric', 'position': '14'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6563b8cb3abc3aea7a00bb45', 'field': 'Balance_binarizer', 'alias': 'Balance_binarizer', 'type': 'real', 'position': '15'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6563b8cb3abc3aea7a00bb46', 'field': 'EstimatedSalary_binarizer', 'alias': 'EstimatedSalary_binarizer', 'type': 'real', 'position': '16'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6563b8cb3abc3aea7a00bb47', 'field': 'Card Type_stringindexer', 'alias': 'Card Type_stringindexer', 'type': 'numeric', 'position': '17'}], 'actual_schema': [{'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6563b8cb3abc3aea7a00bb24', 'field': 'RowNumber', 'alias': 'RowNumber', 'type': 'numeric', 'position': '0'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6563b8cb3abc3aea7a00bb25', 'field': 'CustomerId', 'alias': 'CustomerId', 'type': 'numeric', 'position': '1'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6563b8cb3abc3aea7a00bb26', 'field': 'Surname', 'alias': 'Surname', 'type': 'String', 'position': '2'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6563b8cb3abc3aea7a00bb27', 'field': 'CreditScore', 'alias': 'CreditScore', 'type': 'numeric', 'position': '3'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6563b8cb3abc3aea7a00bb28', 'field': 'Geography', 'alias': 'Geography', 'type': 'String', 'position': '4'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6563b8cb3abc3aea7a00bb29', 'field': 'Gender', 'alias': 'Gender', 'type': 'String', 'position': '5'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6563b8cb3abc3aea7a00bb2a', 'field': 'Age', 'alias': 'Age', 'type': 'numeric', 'position': '6'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6563b8cb3abc3aea7a00bb2b', 'field': 'Tenure', 'alias': 'Tenure', 'type': 'numeric', 'position': '7'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6563b8cb3abc3aea7a00bb2c', 'field': 'Balance', 'alias': 'Balance', 'type': 'real', 'position': '8'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6563b8cb3abc3aea7a00bb2d', 'field': 'NumOfProducts', 'alias': 'NumOfProducts', 'type': 'numeric', 'position': '9'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6563b8cb3abc3aea7a00bb2e', 'field': 'HasCrCard', 'alias': 'HasCrCard', 'type': 'numeric', 'position': '10'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6563b8cb3abc3aea7a00bb2f', 'field': 'IsActiveMember', 'alias': 'IsActiveMember', 'type': 'numeric', 'position': '11'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6563b8cb3abc3aea7a00bb30', 'field': 'EstimatedSalary', 'alias': 'EstimatedSalary', 'type': 'real', 'position': '12'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6563b8cb3abc3aea7a00bb31', 'field': 'Exited', 'alias': 'Exited', 'type': 'numeric', 'position': '13'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6563b8cb3abc3aea7a00bb32', 'field': 'Complain', 'alias': 'Complain', 'type': 'numeric', 'position': '14'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6563b8cb3abc3aea7a00bb33', 'field': 'Satisfaction Score', 'alias': 'Satisfaction Score', 'type': 'numeric', 'position': '15'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6563b8cb3abc3aea7a00bb34', 'field': 'Card Type', 'alias': 'Card Type', 'type': 'String', 'position': '16'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6563b8cb3abc3aea7a00bb35', 'field': 'Point Earned', 'alias': 'Point Earned', 'type': 'numeric', 'position': '17'}], 'server': 'https://colaberry.numtra.com:443', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/', 'delimiter': ',', 'file_type': 'Delimeted', 'filename': 'FECustomerChurn.csv', 'token': '', 'domain': '', 'is_header': 'Use Header Line', 'url': '/FileStore/platform/uploadedSourceFiles/part-00000-5c8344bb-3ce8-4bd2-8b92-ffd68d459202-c000.csv', 'results_url': 'http://colaberry.numtra.com:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
